#Priyanshu Singh 170101049 Assignment 2

###Downloading Corpus

In [ ]:
!wget https://www.dropbox.com/s/1agrh5hdnkqd24c/en_wiki.txt?dl=0        # downloads the dataset
!mv en_wiki.txt?dl=0 en_data.txt

with open("en_data.txt", "r") as file1:                                 # reading the data file
    en_text = file1.read()
en_text = en_text.replace('\n', '')

--2020-11-05 13:14:51--  https://www.dropbox.com/s/1agrh5hdnkqd24c/en_wiki.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1agrh5hdnkqd24c/en_wiki.txt [following]
--2020-11-05 13:14:51--  https://www.dropbox.com/s/raw/1agrh5hdnkqd24c/en_wiki.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd321d1a27d6bab23ff57f6e6c4.dl.dropboxusercontent.com/cd/0/inline/BCk8TWEeH-GDFio1QXMkcfiym2socx3MhP-2gJYLN3Exwl3CGqAHT-0LeEHR7Gjb6RaT4SIov2xm2TF0r1AxdlnCv26OVS-VNx1ltDCi6XZTIG7Ga9hICIgWgB_-IhE4VHA/file# [following]
--2020-11-05 13:14:51--  https://ucd321d1a27d6bab23ff57f6e6c4.dl.dropboxusercontent.com/cd/0/inline/BCk8TWEeH-GDFio1QXMkcfiym2socx3MhP-2gJYLN3Exwl3CGqAHT-0LeEHR7Gjb6RaT4SIov2xm2TF0r1AxdlnCv26OVS-VNx1ltDCi6XZTIG7G

#Trigram Language Model

###```Sentence Segmentation```

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from IPython.display import display, Markdown
import nltk
nltk.download('punkt')  # in order to make "nltk.tokenize.word_tokenize" work

# sentence segmentation using NLTK
sent_tokens_en = nltk.sent_tokenize(en_text)    

# addition of stop words and start words in the corpus
start_str = "START START "
end_str = " STOP"
sent_tokens = []
for sent in sent_tokens_en:
  sent = sent.strip()
  sent = start_str + sent + end_str
  sent_tokens.append(sent)

sent_tokens = sent_tokens[:2000]

del sent_tokens_en

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


###```Preparation of Train+Dev and Test set```

In [ ]:
import random
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

# splitting of data into train and test after random shuffling
random.shuffle(sent_tokens)
no_of_sentences = len(sent_tokens)
train_data_len = 9 * (no_of_sentences // 10)
train_sentences = sent_tokens[:train_data_len]
test_sentences = sent_tokens[train_data_len:]

# some statistics
display(Markdown("#Sentence segmentation using NLTK"))
display(Markdown("##No of sentences after sentence segmentation are: {}".format(len(sent_tokens))))
display(Markdown("##No of sentences in training set: {}".format(len(train_sentences))))
display(Markdown("###Some of the train sentences are:"))
print(train_sentences[:20])
display(Markdown("##No of sentences in test set: {}".format(len(test_sentences))))
display(Markdown("###Some of the test sentences are:"))
print(test_sentences[:20])

#Sentence segmentation using NLTK

##No of sentences after sentence segmentation are: 2000

##No of sentences in training set: 1800

###Some of the train sentences are:

["START START Janill Williams, a young athlete with much promise comes from Gray's Farm, Antigua. STOP", 'START START The US Securities and Exchange Commission has accused the Antigua-based Stanford International Bank, owned by Texas billionaire Allen Stanford, of orchestrating a huge fraud which may have bilked investors of some $8 billion. STOP', 'START START Stars form from dense clouds in the ISM, and the evolutionary processes of stars result in the steady enrichment of the ISM with elements more massive than hydrogen and helium. STOP', 'START START Under this arrangement the author does not pay anything towards the expense of publication. STOP', "START START Even the book review by the editors has more significance than the readership's reception.A standard contract for an author will usually include provision for payment in the form of an advance and royalties. STOP", 'START START the set of "theorems" derived by it, seemed to be identical. STOP', 'START START The senior squad c

##No of sentences in test set: 200

###Some of the test sentences are:

['START START The alternating regular writers were initially Dan Slott, Bob Gale, Marc Guggenheim, Fred Van Lente, and Zeb Wells, joined by a rotation of artists that included Chris Bachalo, Phil Jimenez, Mike McKone, John Romita Jr. and Marcos Martín. STOP', 'START START While almost everyone has experienced anxiety at some point in their lives, most do not develop long-term problems with anxiety.The behavioral effects of anxiety may include withdrawal from situations which have provoked anxiety in the past. STOP', 'START START The electron can change its state to a higher energy level by absorbing a photon with sufficient energy to boost it into the new quantum state. STOP', 'START START The neutron was discovered in 1932 by the English physicist James Chadwick.In the Standard Model of physics, electrons are truly elementary particles with no internal structure. STOP', 'START START Overall, the Earth is about 1.4% aluminium by mass (eighth in abundance by mass). STOP', 'START START H

In [ ]:
# Helper functions to for n-grams analysis
from collections import defaultdict

# returns a list of ngrams
def extract_ngrams(tokens, num):
    n_grams = ngrams(tokens, num)
    return [ ' '.join(grams) for grams in n_grams]

# returns a dict of ngrams
def get_frequency(tokens,num):
  n_grams = extract_ngrams(tokens,num)
  freq = defaultdict(int)
  for i in n_grams:
    freq[i] = freq.get(i,0) + 1
  return freq

##```Linear Interpolation```



In [ ]:
import numpy as np

# returns the probability of a sentence as calculated by a trigram markov model with linear interpolation
def prob_sentence_interpolation(tokens, lambda1, lambda2, lambda3):
  
  modified_tokens = []
  
  # replace less frequent words with "UNK" token
  for token in tokens:
    if freq_unigrams.get(token, 0) < 5:
      token = "UNK"
    modified_tokens.append(token)

  # to avoid underflow, we calculate log probabilities
  log_prob = 0
  for i, val in enumerate(modified_tokens):
    if i > 1:
      str1 = modified_tokens[i-2] + " " + modified_tokens[i-1] + " " + modified_tokens[i]               # (u,v,w) trigram
      str2 = modified_tokens[i-1] + " " + modified_tokens[i]                                            # (v,w) bigram
      cnt1 = freq_trigrams.get(str1, 0)
      cnt2 = freq_bigrams.get(str2, 1)
      cnt3 = freq_unigrams.get(modified_tokens[i-1], 1)
      cnt4 = freq_unigrams.get(modified_tokens[i], 0)
      linear_inter = lambda1 * (cnt1 / cnt2) + lambda2 * (cnt2 / cnt3) + lambda3 * (cnt4 / len(modified_train_tokens))
      if linear_inter == 0:
        continue
      max_likelihood_prob = np.log(linear_inter)
      log_prob += max_likelihood_prob

  return log_prob

# calculates the likelihood value which is used for finding optimal lambda's for linear interpolation
# Likelihood value -> summation(c'(u,v,w) * log(lambda1*prob_trigram + lambda2*prob_bigram + lambda3*prob_unigram)) where c'(u,v,w) is count in dev set
def calc_likelihood_interpolation(lambda1, lambda2, lambda3):
  ans = 0
  for key, value in freq_trigrams.items():
    res = key.split()
    val = freq_trigrams_dev.get(key, 0)
    str1 = res[0] + " " + res[1]
    val1 = lambda1 * (value / freq_bigrams.get(str1, 1))
    val2 = lambda2 * (freq_bigrams.get(str1, 0) / freq_unigrams.get(res[1], 1))
    val3 = lambda3 * (freq_unigrams.get(res[2], 0) / len(modified_train_tokens))
    ans += np.log(val1 + val2 + val3) * val

  return ans

# returns the perplexity over Dev set by doing a grid search for optimal lambda calculation
def calc_perplexity_interpolation(sents):
  m = len(sents)
  n = 0
  best_lambda1 = 0
  best_lambda2 = 0
  best_lambda3 = 0
  max_log_likelihood = -100000000000

  # doing a grid search for lambda, starting from 0.1 to 1.0 with steps of size 0.1
  for lambda1 in range(1, 11):
    for lambda2 in range(1, 11):
      lambda3 = 10 - lambda1 - lambda2
      lambda1 = lambda1 / 10
      lambda2 = lambda2 / 10
      lambda3 = lambda3 / 10
      if lambda3 >= 0:
        curr_log_likelihood = calc_likelihood_interpolation(lambda1, lambda2, lambda3)
        if max_log_likelihood < curr_log_likelihood:
          max_log_likelihood = curr_log_likelihood
          best_lambda1 = lambda1
          best_lambda2 = lambda2
          best_lambda3 = lambda3
  
  # Calculation of perplexity for the whole test corpus
  # Reference:- https://stats.stackexchange.com/a/143638
  prob_sum = 0
  for sent in sents:
    tokens = nltk.word_tokenize(sent)
    n += len(tokens)
    prob_sum += prob_sentence_interpolation(tokens, best_lambda1, best_lambda2, best_lambda3)
  
  ans = -prob_sum / n
  ans = pow(math.e, ans)
  return ans, best_lambda1, best_lambda2, best_lambda3, max_log_likelihood

##```Discounting n-grams```

In [ ]:
# returns the discounted probabilities of bigram tokens as discussed in slides
def discountBigrams(sents, beta):
  discounted_probs = defaultdict(int)

  for key, val in freq_unigrams.items():
    missing_mass = 0
    cum_weight = 0

    for token in freq_unigrams.keys():
      new_key = key + " " + token
      if freq_bigrams.get(new_key, 0) > 0:
        temp = freq_bigrams[new_key] - beta
        missing_mass += temp / val
        discounted_probs[new_key] = temp / val
      else:
        cnt = freq_unigrams.get(token, 0)
        cum_weight += cnt
      
    missing_mass = 1 - missing_mass
    for token in freq_unigrams.keys():
      new_key = key + " " + token
      if freq_bigrams.get(new_key, 0) == 0:
        cnt = freq_unigrams.get(token, 0)
        discounted_probs[new_key] = missing_mass * (cnt / cum_weight)

  return discounted_probs 

# returns the discounted probabilities of trigram tokens using discounted bigram probabilities as discussed in slides
def discountTrigrams(sents, beta, discounted_bigrams_prob):
  discounted_probs = defaultdict(int)

  for key, val in freq_bigrams.items():
    missing_mass = 0
    cum_weight = 0

    for token in freq_unigrams.keys():
      new_key = key + " " + token
      if freq_trigrams.get(new_key, 0) > 0:
        temp = freq_trigrams[new_key] - beta
        missing_mass += temp / val
        discounted_probs[new_key] = temp / val
      else:
        res = key.split()
        temp = res[1]
        temp = temp + " " + token
        cnt = discounted_bigrams_prob.get(temp, 0)
        cum_weight += cnt
    
    missing_mass = 1 - missing_mass
    for token in freq_unigrams.keys():
      new_key = key + " " + token
      if freq_trigrams.get(new_key, 0) == 0:
        res = key.split()
        temp = res[1]
        temp = temp + " " + token
        cnt = discounted_bigrams_prob.get(temp, 0)
        discounted_probs[new_key] = missing_mass * (cnt / cum_weight)
  
  return discounted_probs

##```Perplexity and likelihood calculations for discounting```

In [ ]:
# returns the probability of a sentence as calculated by a trigram markov model with discounting
def prob_sentence_discount(tokens, prob_discount_trigrams):
  modified_tokens = []

  # replace less frequent words with "UNK" token
  for token in tokens:
    if freq_unigrams.get(token, 0) < 5:
      token = "UNK"
    modified_tokens.append(token)

  log_prob = 0

  for i, val in enumerate(modified_tokens):
    if i > 1:
      str1 = modified_tokens[i-2] + " " + modified_tokens[i-1] + " " + modified_tokens[i]
      cnt = prob_discount_trigrams.get(str1, 0)
      if cnt == 0:
        continue
      log_prob += np.log(cnt)
  
  return log_prob

# calculates the likelihood on dev set for finding optimal beta for discounting bigrams
def calc_likelihood_discount_bigrams(discounted_bigrams_prob):
  ans = 0
  for key, val in freq_bigrams_dev.items():
      ans += val * np.log(discounted_bigrams_prob.get(key, 1))
  return ans

# calculates the likelihood on dev set for finding optimal beta for discounting trigrams
def calc_likelihood_discount_trigrams(discounted_trigrams_prob):
  ans = 0
  for key, val in freq_trigrams_dev.items():
      ans += val * np.log(discounted_trigrams_prob.get(key, 1))
  return ans

# returns the perplexity over Dev set by doing a grid search for optimal beta calculation
def calc_perplexity_discount(sents, modified_train_tokens):
  m = len(sents)        
  n = 0                 
  best_beta_bigram = 0
  best_beta_trigram = 0
  max_log_likelihood1 = -100000000000
  max_log_likelihood2 = -100000000000

  # doing a grid search for beta of bigram, starting from 0.1 to 1.0 with steps of size 0.1
  for beta1 in range(1, 11):
    beta1 = beta1 / 10
    prob_discount_bigrams = discountBigrams(modified_train_tokens, beta1)
    curr_log_likelihood = calc_likelihood_discount_bigrams(prob_discount_bigrams)
    if max_log_likelihood1 < curr_log_likelihood:
      max_log_likelihood1 = curr_log_likelihood
      best_beta_bigram = beta1
    del prob_discount_bigrams
  
  prob_discount_bigrams = discountBigrams(modified_train_tokens, best_beta_bigram)

  # doing a grid search for beta of trigram, starting from 0.1 to 1.0 with steps of size 0.1
  for beta2 in range(1, 11):
    beta2 = beta2 / 10
    prob_discount_trigrams = discountTrigrams(modified_train_tokens, beta2, prob_discount_bigrams)
    curr_log_likelihood = calc_likelihood_discount_trigrams(prob_discount_trigrams)

    if max_log_likelihood2 < curr_log_likelihood:
      max_log_likelihood2 = curr_log_likelihood
      best_beta_trigram = beta2
    del prob_discount_trigrams

  prob_discount_trigrams = discountTrigrams(modified_train_tokens, best_beta_trigram, prob_discount_bigrams)

  # Calculation of perplexity for the whole test corpus
  # Reference:- https://stats.stackexchange.com/a/143638
  prob_sum = 0
  for sent in sents:
    tokens = nltk.word_tokenize(sent)
    n += len(tokens)
    prob_sum += prob_sentence_discount(tokens, prob_discount_trigrams)
  
  ans = -prob_sum / n
  ans = pow(math.e, ans)
  return ans, best_beta_bigram, best_beta_trigram, max_log_likelihood2

##```Laplace Smoothing```

In [ ]:
# calculates probability of trigrams in training set from the bigram and trigram frequencies of training set
def laplace_probabilities(freq_bigrams, freq_trigrams, total_words):
  laplace_prob = {}
  for key, val in freq_trigrams.items():
    res = key.split()
    temp = res[0] + " " + res[1]
    prob = (val + 1) / (freq_bigrams.get(temp) + total_words)
    laplace_prob[key] = prob

  return laplace_prob

# calculates the log probability(to prevent underflow) of a sentence by trigram model
def prob_sentence_laplace(tokens, laplace_prob):
  modified_tokens = []

  # replace less frequent words with "UNK" token
  for token in tokens:
    if freq_unigrams.get(token, 0) < 5:
      token = "UNK"
    modified_tokens.append(token)

  log_prob = 0

  for i, val in enumerate(modified_tokens):
    if i > 1:
      str1 = modified_tokens[i-2] + " " + modified_tokens[i-1] + " " + modified_tokens[i]
      cnt = laplace_prob.get(str1, 0)
      if cnt == 0:
        cnt = 1 / len(modified_tokens)
      log_prob += np.log(cnt)
  
  return log_prob

# calculates the perplexity on passed corpus
# Reference:- https://stats.stackexchange.com/a/143638
def calc_perplexity_laplace(sents, laplace_prob):
  m = len(sents)
  n = 0
  prob_sum = 0
  for sent in sents:
    tokens = nltk.word_tokenize(sent)
    n += len(tokens)
    prob_sum += prob_sentence_laplace(tokens, laplace_prob)
  
  ans = -prob_sum / n
  ans = pow(math.e, ans)
  return ans

##```Functions to calculate perplexity on test set```

In [ ]:
# calculates the perplexity on test set using linear interpolation
def test_perplexity_interpolation(sents, lambda1, lambda2, lambda3):
  m = len(sents)        
  n = 0
  prob_sum = 0
  for sent in sents:
    tokens = nltk.word_tokenize(sent)
    n += len(tokens)
    prob_sum += prob_sentence_interpolation(tokens, lambda1, lambda2, lambda3)
  
  ans = -prob_sum / n
  ans = pow(math.e, ans)
  return ans

# calculates the perplexity on test set using discounting
def test_perplexity_discount(sents, modified_train_tokens, best_beta_bigram, best_beta_trigram):
  m = len(sents)        
  n = 0
  prob_discount_bigrams = discountBigrams(modified_train_tokens, best_beta_bigram)
  prob_discount_trigrams = discountTrigrams(modified_train_tokens, best_beta_trigram, prob_discount_bigrams)

  prob_sum = 0
  for sent in sents:
    tokens = nltk.word_tokenize(sent)
    n += len(tokens)
    prob_sum += prob_sentence_discount(tokens, prob_discount_trigrams)
  
  ans = -prob_sum / n
  ans = pow(math.e, ans)
  return ans

##```Results```

In [ ]:
from nltk.util import ngrams
import math
import numpy as np

# making 5 different train and dev sets, finding optimal hyperparameters of trigram model and then evaluating the test set for perplexity
for i in range(5):
  temp = train_sentences
  random.shuffle(train_sentences)                             # random shuffling
  no_of_sentences = len(train_sentences)
  train_data_len = 9 * (no_of_sentences // 10)                # 90:10 split for training and dev set
  dev_sentences = train_sentences[train_data_len:]
  train_sentences = train_sentences[:train_data_len]

  if i == 0:
    display(Markdown("##No of sentences in training set: {}".format(len(train_sentences))))
    display(Markdown("##No of sentences in Dev set: {}".format(len(dev_sentences))))

  # generating the training text from training sentences
  train_text = ""
  for sent in train_sentences:
    train_text += sent
    train_text += "\n"
  train_text = train_text[:-1]

  # generating the dev text from dev sentences
  dev_text = ""
  for sent in dev_sentences:
    dev_text += sent
    dev_text += "\n"
  dev_text = dev_text[:-1]

  # generating train and dev tokens
  train_tokens = nltk.word_tokenize(train_text)
  dev_tokens = nltk.word_tokenize(dev_text)
  freq_unigrams = get_frequency(train_tokens, 1)

  # replacing less frequent words with <UNK> token in training set
  modified_train_tokens = []
  for token in train_tokens:
    if freq_unigrams.get(token, 0) < 5:
      token = "UNK"
    modified_train_tokens.append(token)

  # replacing less frequent words with <UNK> token in dev set
  modified_dev_tokens = []
  for token in dev_tokens:
    if freq_unigrams.get(token, 0) < 5:
      token = "UNK"
    modified_dev_tokens.append(token)

  del freq_unigrams

  # generating frequency dict of n-grams from train set and dev set
  freq_unigrams = get_frequency(modified_train_tokens, 1)
  freq_bigrams = get_frequency(modified_train_tokens, 2)
  freq_trigrams = get_frequency(modified_train_tokens, 3)
  freq_bigrams_dev = get_frequency(modified_dev_tokens, 2)
  freq_trigrams_dev = get_frequency(modified_dev_tokens, 3)
  laplace_prob = laplace_probabilities(freq_bigrams, freq_trigrams, len(modified_train_tokens))

  # various perplexity calculations on dev set
  perplexity_with_interpolation, best_lambda1, best_lambda2, best_lambda3, L_max_interpolation = calc_perplexity_interpolation(dev_sentences)
  perplexity_with_discount, best_beta_bigram, best_beta_trigram, L_max_discount = calc_perplexity_discount(dev_sentences, modified_train_tokens)
  perplexity_with_laplace = calc_perplexity_laplace(dev_sentences, laplace_prob)

  print('\033[90m' + "###### Iteration", i+1, "######" + '\033[0m')
  print('\033[93m' + "On development set" + '\033[0m')
  print("------------------------- Linear Interpolation ---------------------------")
  print("Optimal log likelihood : ", L_max_interpolation)
  print("Optimal lambdas : ", best_lambda1, best_lambda2, best_lambda3)
  print("Perplexity : ", perplexity_with_interpolation)

  print("------------------------- Discounting ---------------------------")
  print("Optimal log likelihood : ", L_max_discount)
  print("Optimal beta for bigram discounting: ", best_beta_bigram)
  print("Optimal beta for trigram discounting: ", best_beta_trigram)
  print("Perplexity : ", perplexity_with_discount)

  print("------------------------- Laplace smoothing ---------------------------")
  print("Perplexity : ", perplexity_with_laplace)
  print("\n")

  # perplexity calculation on test set
  perplexity_test_interpolation = test_perplexity_interpolation(test_sentences, best_lambda1, best_lambda2, best_lambda3)
  perplexity_test_discount = test_perplexity_discount(test_sentences, modified_train_tokens, best_beta_bigram, best_beta_trigram)
  perplexity_test_laplace = calc_perplexity_laplace(test_sentences, laplace_prob)

  print('\033[93m' + "On test set" + '\033[0m')
  print("Peplexity with Linear Interpolation : ", perplexity_test_interpolation)
  print("Peplexity with Discounting: ", perplexity_test_discount)
  print("Peplexity with Laplace Smoothing: ", perplexity_test_laplace)
  print("\n\n")

  train_sentences = temp

  del freq_unigrams
  del freq_bigrams
  del freq_trigrams
  del freq_trigrams_dev
  del freq_bigrams_dev
  del train_text
  del dev_text
  del modified_train_tokens
  del modified_dev_tokens
  del train_tokens
  del dev_tokens

##No of sentences in training set: 1620

##No of sentences in Dev set: 180

###### Iteration 1 ######
On development set
------------------------- Linear Interpolation ---------------------------
Optimal log likelihood :  -6193.2823359387185
Optimal lambdas :  0.9 0.1 0.0
Perplexity :  36.42104929798983
------------------------- Discounting ---------------------------
Optimal log likelihood :  -20632.57895309312
Optimal beta for bigram discounting:  0.6
Optimal beta for trigram discounting:  0.7
Perplexity :  20.652984502166873
------------------------- Laplace smoothing ---------------------------
Perplexity :  339.1967090168631


On test set
Peplexity with Linear Interpolation :  34.27061526816665
Peplexity with Discounting:  20.19779842093701
Peplexity with Laplace Smoothing:  354.0467516907671



###### Iteration 2 ######
On development set
------------------------- Linear Interpolation ---------------------------
Optimal log likelihood :  -6291.9077446398605
Optimal lambdas :  0.9 0.1 0.0
Perplexity :  36.984938329389394
------------------------- Discount